## Create CNN and Optimize using Keras Tuner


https://keras.io/guides/keras_tuner/getting_started/

In [14]:
## pip install keras-tuner

In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [16]:
## Loading fashion dataset
fashion_mnist = keras.datasets.fashion_mnist

In [17]:
(train_images,train_labels), (test_images,test_labels) = fashion_mnist.load_data()

## lets scale the images

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images = train_images/255.0
test_images = test_images/255.0

In [20]:
train_images[0].shape

(28, 28)

In [21]:
## scaled image
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [24]:
## image should be in 4d for the cnn model

## 28,28 because we know the dimensions is 28*28
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)


https://stackoverflow.com/questions/43237124/what-is-the-role-of-flatten-in-keras

In [37]:
def build_model(hp):
    model = keras.Sequential([

        keras.layers.Conv2D(
            filters = hp.Int('conv_1_filter',min_value= 32,max_value=128,step=16),
            kernel_size = hp.Choice('conv_1_kernel',values=[3,5]), ## 3 or 5 filters
            activation='relu',
            input_shape=(28,28,1)
        ),

        keras.layers.Conv2D(
            filters = hp.Int('conv_2_filter',min_value= 32,max_value=64,step=16),
            kernel_size = hp.Choice('conv_1_kernel',values=[3,5]), ## 3 or 5 filters
            activation='relu'
        ),

        keras.layers.Flatten(),
        keras.layers.Dense(
            units = hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
            activation='relu',
        ),

        keras.layers.Dense(10,activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[0.01,0.001])),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model                

In [38]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [40]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',
                max_trials=5,directory='output',project_name='Mnist Fashion')

In [41]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.2)

Trial 8 Complete [00h 00m 25s]
val_accuracy: 0.9048333168029785

Best val_accuracy So Far: 0.9167500138282776
Total elapsed time: 00h 04m 21s
INFO:tensorflow:Oracle triggered exit


In [47]:
## getting top 2 models
model = tuner_search.get_best_models(num_models=2)[0] ## top 1st model 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,690,682
Trainable params: 2,690,682
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.2,initial_epoch=3)
## already the model is trained till 3 epochs, see above code
## so it will directly start from the 4th epoch

Train on 48000 samples, validate on 12000 samples
Epoch 4/10
48000/48000 [==============================] - 10s 212us/sample - loss: 0.1183 - accuracy: 0.9566 - val_loss: 0.2637 - val_accuracy: 0.9167
Epoch 5/10
48000/48000 [==============================] - 10s 209us/sample - loss: 0.0774 - accuracy: 0.9717 - val_loss: 0.2962 - val_accuracy: 0.9139
Epoch 6/10
48000/48000 [==============================] - 11s 222us/sample - loss: 0.0542 - accuracy: 0.9800 - val_loss: 0.3647 - val_accuracy: 0.9171
Epoch 7/10
48000/48000 [==============================] - 10s 210us/sample - loss: 0.0367 - accuracy: 0.9868 - val_loss: 0.4406 - val_accuracy: 0.9130
Epoch 8/10
48000/48000 [==============================] - 10s 212us/sample - loss: 0.0285 - accuracy: 0.9898 - val_loss: 0.4656 - val_accuracy: 0.9124
Epoch 9/10
48000/48000 [==============================] - 10s 205us/sample - loss: 0.0246 - accuracy: 0.9915 - val_loss: 0.5268 - val_accuracy: 0.9114
Epoch 10/10
48000/48000 [===================